In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2
import hippo3 as hippo
from pathlib import Path
from tqdm import tqdm
import pandas as pd
import numpy as np
import molparse as mp

In [ ]:
import logging
logger = logging.getLogger()
logger.setLevel(level=logging.INFO)

In [ ]:
!rm -v chem_test.sqlite

In [ ]:
animal = hippo.HIPPO('chem_test','chem_test.sqlite')

In [ ]:
base_df = pd.read_csv('../wt_designs_MASTER_COMPLETE_FINAL_29Feb.csv')

In [ ]:
from hippo3.tools import inchikey_from_smiles

for i,row in tqdm(base_df.iterrows()):
    
    # if i < 4:
        # continue
        
    if i > 20:
        break
    
    # logger.title(f'{i=}')
    
    # print(row)
    
    j_max = row['no-steps']
    
    for j in range(j_max):
        
        j += 1
        
        s1 = row[f'reactant-1-{j}']
        s2 = row[f'reactant-2-{j}']
        s3 = row[f'reaction-product-smiles-{j}']

        # print(s1,s2,s3)
        
        # logger.var('j',j)
        # logger.var('s1',s1)
        # logger.var('s2',s2)
        # logger.var('s3',s3)
        
        # add the reactants
        
        c1 = animal.db.insert_compound(smiles=s1, warn_duplicate=False)
        # if c1: logger.info(f'added C{c1}')
        
        if isinstance(s2,float) and np.isnan(s2):
            c2 = None
            s2 = None
        else:
            c2 = animal.db.insert_compound(smiles=s2, warn_duplicate=False)
            # if c2: logger.info(f'added C{c2}')
            
        if j == j_max:
            tags = ['base']
        else:
            tags = ['intermediate']
            
        c3 = animal.db.insert_compound(smiles=s3, tags=tags, warn_duplicate=False)
        # if c3: logger.info(f'added C{c3}')
                        
        if c1 is None:
            c1 = animal.compounds[inchikey_from_smiles(s1)]
                
        if c2 is None:
            if s2 is not None:
                c2 = animal.compounds[inchikey_from_smiles(s2)]
        
        if c3 is None:
            c3 = animal.compounds[inchikey_from_smiles(s3)]
        
        r1 = animal.db.insert_reaction(type=row[f'reaction-name-{j}'], product=c3)
        # if r1: logger.info(f'Added R{r1}')
        
        animal.db.insert_reactant(compound=c1, reaction=r1, amount=1)
        # if r1: logger.info(f'Added R{r1} reactant: C{c1}')

        if c2:
            animal.db.insert_reactant(compound=c2, reaction=r1, amount=1)
            # if r1: logger.info(f'Added R{r1} reactant: C{c2}')
        
    # c3 = animal.db.insert_compound(smiles=s3, tags=['base'])
    # if c3: logger.info(f'Added product C{c3}')
    
    # logger.title(f'{animal.compounds[inchikey_from_smiles(s3)]} {c3}')

    # break

In [ ]:
s = 'Cc1ccc(NC(=O)CO)cc1NC(=O)CC#N'
m = mp.rdkit.mol_from_smiles(s)
# animal.db.insert_compound(smiles=s)
m

In [ ]:
animal.db.query_similarity(s, threshold=0.8, return_similarity=True)

In [ ]:
animal.compounds.get_by_tag('base').ids

In [ ]:
recipe = animal.compounds[6,10,14].get_recipe(amount=[20,20,20,20])
# recipe = animal.compounds[6].reactions[0].get_recipe(amount=20)

In [ ]:
def color_mapper():
    mapper = {}
    
    for comp in animal.compounds.get_by_tag('base'):
        mapper[str(comp)] = (0.0,1.0,0.0)
        
    for comp in animal.compounds.get_by_tag('intermediate'):
        mapper[str(comp)] = (0.0,0.5,1.0)
        
    return mapper

In [ ]:
recipe.summary()

In [ ]:
recipe.draw(color_mapper=color_mapper(), node_size=400)
# recipe.draw()

In [ ]:
recipe.sankey()

In [ ]:
animal.compounds[40].mol

In [ ]:
mp.rdkit.draw_grid([s, animal.compounds[40].smiles], align_substructure=True)

In [ ]:
'CC(C)(C)OC(=O)Nc1cc(N)c(C)cc1'

In [ ]:
# animal.db.delete_where(table='compound', key='id', value=11)

In [ ]:
animal.compounds[11].mol

In [ ]:
animal.compounds.get_by_tag('base')

In [ ]:
animal.reactions[10].draw()

In [ ]:
animal.reactions[2].draw()

In [ ]:
animal.reactions[3].draw()

In [ ]:
recipe = animal.compounds[-1].reactions[0].get_recipe()

In [ ]:
for reaction in recipe.reactions:
    print(reaction)

In [ ]:
recipe.draw()

In [ ]:
animal.db.print_table('tag')